## Практическое задание №3 Embedding word2vec fasttext

Задача поиск похожих по эмбеддингам

Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

# пример работы с ним
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

что надо сделать

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [17]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pandas as pd
import numpy as np

In [2]:
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
arr = []
for row in records:
    arr.append(row.title)


## Предобработка

> Преобразуем в датафрейм только заголовок
> На уроке предлагался именно такой вариант

In [3]:

df = pd.DataFrame(arr,columns=['title'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865846 entries, 0 to 865845
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   865415 non-null  object
dtypes: object(1)
memory usage: 6.6+ MB


In [4]:
df.head(3)

,title
0,Японские физики повторили синтез 113-го элемента
1,Times: Россия строит новую авиабазу в Сирии
2,Власти Египта гарантируют безопасность российс...


> Проверка на пропуски

In [22]:
df.isnull().sum()

title    431
dtype: int64

> Удалим пропуски

In [23]:
df=df[df.title.notna()]

> Процедура преобразования в нормальную форму

In [24]:
def preprocess_txt(line, morpher=None, exclude = None, stop_words = None):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in stop_words and i != ""]
    return spls

In [31]:
morpher = MorphAnalyzer()
stop_words = set(get_stop_words("ru"))
exclude = set(string.punctuation)

> Прогоняем на весь датасет

In [ ]:
df['sentences'] = df['title'].apply(lambda x: preprocess_txt(x, morpher=morpher, exclude=exclude, stop_words=stop_words))

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865415 entries, 0 to 865845
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      865415 non-null  object
 1   sentences  865415 non-null  object
dtypes: object(2)
memory usage: 19.8+ MB


In [27]:
df.head(3)

,title,sentences
0,Японские физики повторили синтез 113-го элемента,"[японский, физика, повторить, синтез, 113го, э..."
1,Times: Россия строит новую авиабазу в Сирии,"[times, россия, строить, новый, авиабаза, сирия]"
2,Власти Египта гарантируют безопасность российс...,"[власть, египет, гарантировать, безопасность, ..."


>Сохраним результат предобработки в pickle-файл

In [28]:
df.to_pickle('gazeta.pkl')

In [4]:
df = pd.read_pickle('gazeta.pkl')

In [5]:
df.head(3)

,title,sentences
0,Японские физики повторили синтез 113-го элемента,"[японский, физика, повторить, синтез, 113го, э..."
1,Times: Россия строит новую авиабазу в Сирии,"[times, россия, строить, новый, авиабаза, сирия]"
2,Власти Египта гарантируют безопасность российс...,"[власть, египет, гарантировать, безопасность, ..."


In [6]:
Word2Vec?

In [10]:
sentences = df.sentences.to_numpy()

In [11]:
sentences[0]

['японский', 'физика', 'повторить', 'синтез', '113го', 'элемент']

In [12]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [18]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

## Построение ембеддинга

In [19]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0


for line in sentences:
    n_w2v = 0
    n_ft = 0
    question = line
    # index_map[counter] = spls[1]

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [78]:
def get_response(question, index, model, df):
    question = preprocess_txt(TEXT, morpher=morpher, exclude=exclude, stop_words=stop_words)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [df.iloc[[i]].title.values[0] for i in answers]


## Тестирование

In [79]:
TEXT = "курс доллара"
get_response(TEXT, w2v_index, modelW2V,df)

['Три легионера «Нефтехимика» бастуют из-за курса евро',
 'Клубы РФПЛ предложили футболистам зафиксировать курс доллара на отметке 45 рублей',
 'Сорокин: бюджет оргкомитета «Россия-2018» не зависит от курса валют',
 'Прядкин: падение курса валют не должно сказаться на РФПЛ',
 'ФК «Локомотив» не откажется от сборов в Испании, несмотря на падение курса рубля']

In [86]:
TEXT = "курс доллара"
get_response(TEXT, ft_index, modelFT,df)

['Прядкин: падение курса валют не должно сказаться на РФПЛ',
 'Три легионера «Нефтехимика» бастуют из-за курса евро',
 'Дель Боске: Касильяс намерен выступать за сборную минимум до Евро 2016',
 'Уткин: перспективы ЦСКА в ЛЧ зависят от здоровья Думбии и от жеребьевки ',
 'Копейкин: у ЦСКА должно быть преимущество в центре поля']

In [88]:
TEXT = "рыночная экономика"
get_response(TEXT, w2v_index, modelW2V,df)

['Капелло: для меня спорт выше политики',
 'Радимов: кризис в «Зените» действительно был',
 'Самедов: «Локомотиву» надо выбираться из кризиса',
 'Игонин: ставка на легионеров в «Зените» очевидна',
 'Анчелотти: необходимо работать, чтобы наш рост продолжался']

In [89]:
TEXT = "рыночная экономика"
get_response(TEXT, ft_index, modelFT,df)

['Собкин: российскому теннису нужны финансовые вливания',
 '«Экономически мы очень зависимы»',
 'Инфантино уверен, что достоин Нобелевской премии по экономике за идею финансового фэйр-плей',
 'Значков: Липницкая особенно сильна в произвольной программе',
 'Аникеева: России необходима единая стратегия развития женского баскетбола']